### **Google Drive Bağlantısı**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **Kurulumlar**

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

### **Veriyi Zipten Çıkarma**


In [ ]:
!unzip "/content/drive/MyDrive/f1.zip"

### **Veriyi Hazırlama ve Görselleştirme**

In [ ]:
data = DataLoader.from_folder("/content/f1")
train_data, test_data = data.split(0.8)
validation_data, test_data1 = test_data.split(0.5)

In [ ]:
train_data

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

### **Model Eğitimi**

In [ ]:
model = image_classifier.create(train_data,model_spec=model_spec.get('efficientnet_lite4'),
                                batch_size=16,epochs=300,
                                learning_rate=0.001,
                                validation_data=validation_data,
                                warmup_steps=20)

In [ ]:
model.summary()

### **Model Testi**

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(10)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

### **Export Alma ve TFLite Modelini Test Etme**


In [ ]:
model.export(export_dir='model_clas.tflite')

In [ ]:
model.evaluate_tflite('/content/model_clas.tflite/model.tflite', test_data)

### **TensorFLow Lite Modelinde Eğitim Sonrası Hassasiyeti Özelleştirme**


In [ ]:
config = QuantizationConfig.for_float16()

In [ ]:
model.export(export_dir='model_fp16.tflite', tflite_filename='model_fp16.tflite', quantization_config=config)